In [1]:
# STEP 0: Install required libraries
!pip install transformers datasets stanza nltk --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.6 MB/s eta 0:00:00 0:00:01


In [2]:
# STEP 1: Import libraries
import os
import json
import pandas as pd
import numpy as np
import sqlite3
import nltk
import stanza
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, AdamW
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm

nltk.download('punkt')
stanza.download('ar')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import json

rows = []
with open("/kaggle/input/txttosql-nlp/AR_spider.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        rows.append(json.loads(line.strip()))

import pandas as pd
df = pd.DataFrame(rows)


In [4]:
print(df.head())


                                            question  \
0  How many heads of the departments are older th...   
1  List the name, born state and age of the heads...   
2  List the creation year, name and budget of eac...   
3  What are the maximum and minimum budget of the...   
4  What is the average number of employees of the...   

                                               query  \
0         SELECT count(*) FROM head WHERE age  >  56   
1  SELECT name ,  born_state ,  age FROM head ORD...   
2  SELECT creation ,  name ,  budget_in_billions ...   
3  SELECT max(budget_in_billions) ,  min(budget_i...   
4  SELECT avg(num_employees) FROM department WHER...   

                                              arabic                  db_id  
0  كم عدد رؤساء الأقسام الذين تزيد أعمارهم عن 56 ...  department_management  
1  اعرض قائمة بأسماء رؤساء الأقسام، مكان ميلادهم،...  department_management  
2  اعرض قائمة بسنوات الإنشاء، وأسماء وميزانيات كل...  department_management  
3             

In [5]:
# STEP 3: Preprocess Arabic text
nlp = stanza.Pipeline('ar', processors='tokenize,lemma')
stopwords = {'في', 'من', 'على', 'و', 'عن', 'إلى', 'مع', 'التي', 'الذي', 'هذا', 'أن', 'ما', 'هو', 'هي'}

def preprocess_arabic(text):
    doc = nlp(text)
    return ' '.join([w.lemma for s in doc.sentences for w in s.words if w.lemma not in stopwords])

In [6]:

# Apply preprocessing + task prefix
df['input_text'] = df['arabic'].apply(lambda x: "ترجم إلى SQL: " + preprocess_arabic(x))
df['target_text'] = df['query']

In [7]:
# STEP 4: Split data
X = df['input_text'].tolist()
y = df['target_text'].tolist()
db_ids = df['db_id'].tolist()

X_train, X_temp, y_train, y_temp, db_train, db_temp = train_test_split(X, y, db_ids, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test, db_val, db_test = train_test_split(X_temp, y_temp, db_temp, test_size=0.5, random_state=42)

In [8]:
# STEP 5: Load model and tokenizer
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
# STEP 6: Tokenize
max_len = 128

def tokenize_data(X, y):
    encodings = tokenizer(X, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
    targets = tokenizer(y, padding=True, truncation=True, max_length=max_len, return_tensors="pt")
    return encodings, targets

train_enc, train_tar = tokenize_data(X_train, y_train)
val_enc, val_tar = tokenize_data(X_val, y_val)

In [10]:
# STEP 7: Create DataLoaders
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = TensorDataset(train_enc['input_ids'], train_enc['attention_mask'], train_tar['input_ids'])
val_dataset = TensorDataset(val_enc['input_ids'], val_enc['attention_mask'], val_tar['input_ids'])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

In [11]:
# STEP 8: Training
model.to(dev)
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 8

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch in loop:
        input_ids, attention_mask, labels = [b.to(dev) for b in batch]
        labels[labels == tokenizer.pad_token_id] = -100

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 8/8: 100%|██████████| 1120/1120 [05:40<00:00,  3.28it/s, loss=2.27] 


In [19]:
# STEP 9: Inference function
def generate_sql(question, table_names=None):
    model.eval()
    schema_part = f" ### الجداول: {', '.join(table_names)}" if table_names else ""
    prompt = "ترجم إلى SQL: " + preprocess_arabic(question) + schema_part
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=max_len).to(dev)
    outputs = model.generate(**inputs, max_length=max_len, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [23]:
# STEP 10: Build test_data list from X_test, y_test, db_test
test_data = []
for i in range(len(X_test)):
    row = df[df['input_text'] == X_test[i]].iloc[0]
    test_data.append({
        "question": row['arabic'],
        "query": y_test[i],
        "db_id": db_test[i],
        "table_names": row.get('table_names', [])
    })

In [29]:
# STEP 11: Evaluation using test_data format
smoothie = SmoothingFunction().method4
def execute_sql(db_id, sql_query):
    db_path = f"/kaggle/input/txttosql-nlp/database/{db_id}.sqlite"
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(sql_query)
        result = cursor.fetchall()
        conn.close()
        return result
    except Exception as e:
        return f"Error: {str(e)}"

# Metrics tracking
exact_match_count = 0
execution_match_count = 0
bleu_scores = []
total_samples = len(test_data)

for idx, entry in enumerate(test_data[:50]):
    print(f"Sample {idx + 1}/{min(50, len(test_data))}")

    print(f"🔸 Question: {entry['question']}")
    print(f"📁 DB: {entry['db_id']}.sqlite")
    print(f"✅ Ground Truth SQL: {entry['query']}")
    arabic_question = entry["question"]
    correct_sql = sqlparse.format(entry["query"], reindent=True, keyword_case="upper")
    db_id = entry["db_id"]

    predicted_sql = generate_sql(arabic_question, entry.get("table_names", [])).strip()
    print(f"✨ Predicted SQL: {predicted_sql}")

    # Exact Match Accuracy
    if predicted_sql.strip() == correct_sql.strip():
        exact_match_count += 1

    # Execution Accuracy
    correct_result = execute_sql(db_id, correct_sql)
    predicted_result = execute_sql(db_id, predicted_sql)
    if correct_result == predicted_result:
        print("✅ SQL execution matched.")
        execution_match_count += 1
    else:
        print("❌ Execution mismatch or error occurred.")
        print("Correct Result:", correct_result)
        print("Predicted Result:", predicted_result)

    # BLEU Score (SQL as tokenized words)
    reference_tokens = correct_sql.split()
    predicted_tokens = predicted_sql.split()
    bleu_scores.append(sentence_bleu([reference_tokens], predicted_tokens, smoothing_function=smoothie))

# Final Metrics
exact_match_acc = exact_match_count / total_samples
execution_acc = execution_match_count / total_samples
average_bleu = sum(bleu_scores) / total_samples

# Display Results
print(f"✅ Exact Match Accuracy: {exact_match_acc:.4f}")
print(f"✅ Execution Accuracy: {execution_acc:.4f}")
print(f"✅ Average BLEU Score: {average_bleu:.4f}")

Sample 1/50
🔸 Question: ما هي أوضاع اللاعبين مع متوسط عدد النقاط المُسجَّلة من قبل اللاعبين في تلك الوضع أكبر من 20؟
📁 DB: sports_competition.sqlite
✅ Ground Truth SQL: SELECT POSITION ,  avg(Points) FROM player GROUP BY POSITION
✨ Predicted SQL: SELECT T1.name , T1.name , T1.name , T1.name FROM exhibition AS T1 JOIN film AS T2 ON T1.film_id = T2.film_id WHERE T1.film_id = T2.film_id GROUP BY T1.film_id ORDER BY count(*) DESC LIMIT 1
✅ SQL execution matched.
Sample 2/50
🔸 Question: [Question] أعط المدينة التي يعيش فيها الطالب الذي يحمل اسم العائلة Kim.
📁 DB: allergy_1.sqlite
✅ Ground Truth SQL: SELECT city_code FROM Student WHERE LName  =  "Kim"
✨ Predicted SQL: SELECT T1.name FROM employees AS T1 JOIN customers AS T2 ON T1.customer_id = T2.customer_id WHERE T1.customer_id = T2.customer_id GROUP BY T1.customer_id ORDER BY count(*) DESC LIMIT 1
✅ SQL execution matched.
Sample 3/50
🔸 Question: فرز قائمة جميع الأسماء الأولى والألقاب للكتّاب ترتيباً أبجدياً حسب الألقاب.
📁 DB: icfp_1.sqlite

In [30]:
# STEP 11: Evaluation using test_data format
smoothie = SmoothingFunction().method4
def execute_sql(db_id, sql_query):
    db_path = f"/kaggle/input/txttosql-nlp/database/{db_id}.sqlite"
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(sql_query)
        result = cursor.fetchall()
        conn.close()
        return result
    except Exception as e:
        return f"Error: {str(e)}"

# Metrics tracking
exact_match_count = 0
execution_match_count = 0
bleu_scores = []
total_samples = len(test_data)

for idx, entry in enumerate(test_data):
    print(f"Sample {idx + 1}/{len(test_data)}")

    print(f"🔸 Question: {entry['question']}")
    print(f"📁 DB: {entry['db_id']}.sqlite")
    print(f"✅ Ground Truth SQL: {entry['query']}")
    arabic_question = entry["question"]
    correct_sql = sqlparse.format(entry["query"], reindent=True, keyword_case="upper")
    db_id = entry["db_id"]

    predicted_sql = generate_sql(arabic_question, entry.get("table_names", [])).strip()
    print(f"✨ Predicted SQL: {predicted_sql}")

    # Exact Match Accuracy
    if predicted_sql.strip() == correct_sql.strip():
        exact_match_count += 1

    # Execution Accuracy
    correct_result = execute_sql(db_id, correct_sql)
    predicted_result = execute_sql(db_id, predicted_sql)
    if correct_result == predicted_result:
        print("✅ SQL execution matched.")
        execution_match_count += 1
    else:
        print("❌ Execution mismatch or error occurred.")
        print("Correct Result:", correct_result)
        print("Predicted Result:", predicted_result)

    # BLEU Score (SQL as tokenized words)
    reference_tokens = correct_sql.split()
    predicted_tokens = predicted_sql.split()
    bleu_scores.append(sentence_bleu([reference_tokens], predicted_tokens, smoothing_function=smoothie))

# Final Metrics
exact_match_acc = exact_match_count / total_samples
execution_acc = execution_match_count / total_samples
average_bleu = sum(bleu_scores) / total_samples

# Display Results
print(f"✅ Exact Match Accuracy: {exact_match_acc:.4f}")
print(f"✅ Execution Accuracy: {execution_acc:.4f}")
print(f"✅ Average BLEU Score: {average_bleu:.4f}")


Sample 1/960
🔸 Question: ما هي أوضاع اللاعبين مع متوسط عدد النقاط المُسجَّلة من قبل اللاعبين في تلك الوضع أكبر من 20؟
📁 DB: sports_competition.sqlite
✅ Ground Truth SQL: SELECT POSITION ,  avg(Points) FROM player GROUP BY POSITION
✨ Predicted SQL: SELECT T1.name , T1.name , T1.name , T1.name FROM exhibition AS T1 JOIN film AS T2 ON T1.film_id = T2.film_id WHERE T1.film_id = T2.film_id GROUP BY T1.film_id ORDER BY count(*) DESC LIMIT 1
✅ SQL execution matched.
Sample 2/960
🔸 Question: [Question] أعط المدينة التي يعيش فيها الطالب الذي يحمل اسم العائلة Kim.
📁 DB: allergy_1.sqlite
✅ Ground Truth SQL: SELECT city_code FROM Student WHERE LName  =  "Kim"
✨ Predicted SQL: SELECT T1.name FROM employees AS T1 JOIN customers AS T2 ON T1.customer_id = T2.customer_id WHERE T1.customer_id = T2.customer_id GROUP BY T1.customer_id ORDER BY count(*) DESC LIMIT 1
✅ SQL execution matched.
Sample 3/960
🔸 Question: فرز قائمة جميع الأسماء الأولى والألقاب للكتّاب ترتيباً أبجدياً حسب الألقاب.
📁 DB: icfp_1.sql